## Capstone Project The Battle of Neighborhoods (Week 1)

In [1]:
# proposition theme
!pip install geopy
!conda install -c conda-forge folium=0.5.0 --yes 

import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print('Libraries imported.')

Requirement not upgraded as not directly required: geopy in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages
Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    altair:  2.2.2-py_0 conda-forge
    branca:  0.3.0-py_0 conda-forge
    folium:  0.5.0-py_0 conda-forge
    vincent: 0.4.4-py_1 conda-forge

branca-0.3.0-p 100% |################################| Time: 0:00:00 315.32 kB/s
vincent-0.4.4- 100% |################################| Time: 0:00:00 361.72 kB/s
altair-2.2.2-p 100% |################################| Time: 0:00:00 518.42 kB/s
folium-0.5.0-p 100% |################################| Time: 0:00:00  41.77 MB/s
Libraries imported.


### Etape 1 : chargement données Paris

In [2]:
paris_data = pd.read_csv("https://lvdesign.com.fr/coursera_data/paris.csv")
paris_data 

,Lieux,Latitude,Longitude
0,Musée du Louvre,48.860611,2.335450
1,Notre Dame de Paris,48.852968,2.347708
2,Arc de Triomphe,48.873792,2.292833
3,Tour Eiffel,48.858370,2.294481


### Localise Lieux on the map Paris

In [3]:
# Localise Lieux on the map Paris
address = 'Paris, FR'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Paris are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Paris are 48.8566101, 2.3514992.


In [4]:
# create map of Paris using latitude and longitude values
map_Paris = folium.Map(location=[latitude, longitude], zoom_start=14)

# add markers to map
for lat, lng, label in zip(paris_data['Latitude'], paris_data['Longitude'], paris_data['Lieux']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Paris)  
    
map_Paris

### Define Foursquare Credentials and Version¶

In [6]:
#Define Foursquare Credentials and Version
CLIENT_ID = 'XDP05EROIGY2VHJ3RZNRVGCEECJKCBAV0L4JMC4F4SCL0NAF' # your Foursquare ID
CLIENT_SECRET = 'XHKK22SSQYX0REMVWFGNG3GTY5I2HGPPOUERFOEFPIR0YPNO' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: XDP05EROIGY2VHJ3RZNRVGCEECJKCBAV0L4JMC4F4SCL0NAF
CLIENT_SECRET:XHKK22SSQYX0REMVWFGNG3GTY5I2HGPPOUERFOEFPIR0YPNO


### Explore all activities on the Lieux in Paris

In [15]:
def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT=10):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng,            
            v['venue']['name'], 
            v['venue']['id'],
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Lieux', 
                  'Lieux Latitude', 
                  'Lieux Longitude', 
                  'Venue',
                  'Venue id',                
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category'        
                   ]
    
    return(nearby_venues)

In [16]:
paris_data_venues = getNearbyVenues(names=paris_data['Lieux'],
                                   latitudes=paris_data['Latitude'],
                                   longitudes=paris_data['Longitude']
                                  )

Musée du Louvre
Notre Dame de Paris
Arc de Triomphe
Tour Eiffel


In [17]:
print(paris_data_venues.shape)
paris_data_venues

(40, 8)


,Lieux,Lieux Latitude,Lieux Longitude,Venue,Venue id,Venue Latitude,Venue Longitude,Venue Category
0,Musée du Louvre,48.860611,2.335450,Musée du Louvre,4adcda10f964a520af3521e3,48.860847,2.336440,Art Museum
1,Musée du Louvre,48.860611,2.335450,La Vénus de Milo (Vénus de Milo),5864efb745c3ed1e7d88e96d,48.861100,2.335863,Exhibit
2,Musée du Louvre,48.860611,2.335450,Mona Lisa | La Joconde,56f279c4cd10850a585f5e31,48.860139,2.335337,Exhibit
3,Musée du Louvre,48.860611,2.335450,Cour Carrée du Louvre,4c079d740ed3c928b6be797d,48.860360,2.338543,Plaza
4,Musée du Louvre,48.860611,2.335450,Carrousel du Louvre,4adcda1df964a5202e3921e3,48.861764,2.334344,Shopping Mall
5,Musée du Louvre,48.860611,2.335450,Cour Napoléon,5072efe7e4b0c34b5146e7fd,48.861172,2.335088,Plaza
6,Musée du Louvre,48.860611,2.335450,Place du Palais Royal,4b071505f964a520dcf622e3,48.862523,2.336688,Plaza
7,Musée du Louvre,48.860611,2.335450,Café Marly,4adcda04f964a520503221e3,48.861596,2.335676,Café
8,Musée du Louvre,48.860611,2.335450,La Maison du Chocolat,4c21f8fd13c00f4729ab86de,48.861417,2.334132,Chocolate Shop
9,Musée du Louvre,48.860611,2.335450,Mariage Frères,4b9cf29ef964a520318436e3,48.861437,2.334143,Tea Room


In [18]:
# More intersting
#Let's check how many venues were returned for each lieux
paris_data_venues.groupby('Lieux').count()

,Lieux Latitude,Lieux Longitude,Venue,Venue id,Venue Latitude,Venue Longitude,Venue Category
Lieux,,,,,,,
Arc de Triomphe,10,10,10,10,10,10,10
Musée du Louvre,10,10,10,10,10,10,10
Notre Dame de Paris,10,10,10,10,10,10,10
Tour Eiffel,10,10,10,10,10,10,10


In [19]:
#Let's find out how many unique categories can be curated from all the returned venues¶
print('There are {} uniques categories.'.format(len(paris_data_venues['Venue Category'].unique())))

There are 26 uniques categories.


In [20]:
#Analyze Each Lieux
# one hot encoding
paris_onehot = pd.get_dummies(paris_data_venues[['Venue Category']], prefix="", prefix_sep="")

# add Lieux column back to dataframe
paris_onehot['Lieux'] = paris_data_venues['Lieux'] 

# move lieux column to the first column
fixed_columns = [paris_onehot.columns[-1]] + list(paris_onehot.columns[:-1])
paris_onehot = paris_onehot[fixed_columns]

paris_onehot.head()

,Lieux,Art Museum,Bagel Shop,Bookstore,Burger Joint,Café,Champagne Bar,Chocolate Shop,Church,Dessert Shop,Exhibit,French Restaurant,Gift Shop,Hotel,Ice Cream Shop,Italian Restaurant,Japanese Restaurant,Monument / Landmark,Nightclub,Plaza,Roof Deck,Scenic Lookout,Shopping Mall,Steakhouse,Tea Room,Theme Park Ride / Attraction,Vegetarian / Vegan Restaurant
0,Musée du Louvre,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Musée du Louvre,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Musée du Louvre,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Musée du Louvre,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
4,Musée du Louvre,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0


In [21]:
# Next, 
# let's group rows by Lieux
# and by taking the mean of the frequency of occurrence of each category
paris_grouped = paris_onehot.groupby('Lieux').mean().reset_index()
paris_grouped

,Lieux,Art Museum,Bagel Shop,Bookstore,Burger Joint,Café,Champagne Bar,Chocolate Shop,Church,Dessert Shop,Exhibit,French Restaurant,Gift Shop,Hotel,Ice Cream Shop,Italian Restaurant,Japanese Restaurant,Monument / Landmark,Nightclub,Plaza,Roof Deck,Scenic Lookout,Shopping Mall,Steakhouse,Tea Room,Theme Park Ride / Attraction,Vegetarian / Vegan Restaurant
0,Arc de Triomphe,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.2,0.0,0.0,0.0,0.1,0.1,0.1,0.1,0.0,0.0,0.1,0.0,0.0,0.1
1,Musée du Louvre,0.1,0.0,0.0,0.0,0.1,0.0,0.1,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3,0.0,0.0,0.1,0.0,0.1,0.0,0.0
2,Notre Dame de Paris,0.0,0.0,0.1,0.1,0.0,0.0,0.0,0.1,0.1,0.0,0.2,0.0,0.0,0.1,0.0,0.1,0.0,0.0,0.1,0.0,0.1,0.0,0.0,0.0,0.0,0.0
3,Tour Eiffel,0.1,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.2,0.1,0.2,0.0,0.1,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0


In [22]:
#Let's print each Lieux along with the top 5 most common venues¶
num_top_venues = 20

for hood in paris_grouped['Lieux']:
    print("----"+hood+"----")
    temp = paris_grouped[paris_grouped['Lieux'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Arc de Triomphe----
                            venue  freq
0                           Hotel   0.2
1   Vegetarian / Vegan Restaurant   0.1
2                      Steakhouse   0.1
3                       Roof Deck   0.1
4                           Plaza   0.1
5                       Nightclub   0.1
6               French Restaurant   0.1
7             Monument / Landmark   0.1
8                      Bagel Shop   0.1
9             Japanese Restaurant   0.0
10   Theme Park Ride / Attraction   0.0
11                       Tea Room   0.0
12                  Shopping Mall   0.0
13                 Scenic Lookout   0.0
14                     Art Museum   0.0
15             Italian Restaurant   0.0
16                      Gift Shop   0.0
17                        Exhibit   0.0
18                   Dessert Shop   0.0
19                         Church   0.0


----Musée du Louvre----
                           venue  freq
0                          Plaza   0.3
1                        Exhibit

In [23]:
#Let's put that into a pandas dataframe
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [24]:
#Now let's create the new dataframe 
#and display the top 10 venues for each Lieux.
num_top_venues = 20

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Lieux']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
lieux_venues_sorted = pd.DataFrame(columns=columns)
lieux_venues_sorted['Lieux'] = paris_grouped['Lieux']

for ind in np.arange(paris_grouped.shape[0]):
    lieux_venues_sorted.iloc[ind, 1:] = return_most_common_venues(paris_grouped.iloc[ind, :], num_top_venues)

lieux_venues_sorted

,Lieux,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
0,Arc de Triomphe,Hotel,Plaza,Bagel Shop,French Restaurant,Monument / Landmark,Nightclub,Vegetarian / Vegan Restaurant,Roof Deck,Steakhouse,Dessert Shop,Tea Room,Bookstore,Burger Joint,Café,Champagne Bar,Chocolate Shop,Church,Exhibit,Scenic Lookout,Gift Shop
1,Musée du Louvre,Plaza,Exhibit,Art Museum,Tea Room,Shopping Mall,Café,Chocolate Shop,French Restaurant,Bagel Shop,Bookstore,Burger Joint,Champagne Bar,Church,Dessert Shop,Vegetarian / Vegan Restaurant,Gift Shop,Theme Park Ride / Attraction,Ice Cream Shop,Italian Restaurant,Japanese Restaurant
2,Notre Dame de Paris,French Restaurant,Bookstore,Burger Joint,Scenic Lookout,Plaza,Japanese Restaurant,Church,Ice Cream Shop,Dessert Shop,Vegetarian / Vegan Restaurant,Bagel Shop,Café,Champagne Bar,Chocolate Shop,Exhibit,Hotel,Gift Shop,Theme Park Ride / Attraction,Italian Restaurant,Monument / Landmark
3,Tour Eiffel,Hotel,French Restaurant,Monument / Landmark,Champagne Bar,Gift Shop,Theme Park Ride / Attraction,Italian Restaurant,Art Museum,Nightclub,Plaza,Bagel Shop,Bookstore,Burger Joint,Café,Tea Room,Chocolate Shop,Church,Dessert Shop,Exhibit,Steakhouse


In [25]:
# set number of clusters
kclusters = 3

paris_grouped_clustering = paris_grouped.drop('Lieux', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(paris_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 2, 1, 0], dtype=int32)

In [26]:
#Let's create a new dataframe that includes the cluster as well as the top 5 venues for each Lieux.

paris_merged = paris_data

# add clustering labels
paris_merged['Cluster Labels'] = kmeans.labels_

paris_merged = paris_merged.join(lieux_venues_sorted.set_index('Lieux'), on='Lieux')
paris_merged 
# check the last columns!

,Lieux,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
0,Musée du Louvre,48.860611,2.335450,0,Plaza,Exhibit,Art Museum,Tea Room,Shopping Mall,Café,Chocolate Shop,French Restaurant,Bagel Shop,Bookstore,Burger Joint,Champagne Bar,Church,Dessert Shop,Vegetarian / Vegan Restaurant,Gift Shop,Theme Park Ride / Attraction,Ice Cream Shop,Italian Restaurant,Japanese Restaurant
1,Notre Dame de Paris,48.852968,2.347708,2,French Restaurant,Bookstore,Burger Joint,Scenic Lookout,Plaza,Japanese Restaurant,Church,Ice Cream Shop,Dessert Shop,Vegetarian / Vegan Restaurant,Bagel Shop,Café,Champagne Bar,Chocolate Shop,Exhibit,Hotel,Gift Shop,Theme Park Ride / Attraction,Italian Restaurant,Monument / Landmark
2,Arc de Triomphe,48.873792,2.292833,1,Hotel,Plaza,Bagel Shop,French Restaurant,Monument / Landmark,Nightclub,Vegetarian / Vegan Restaurant,Roof Deck,Steakhouse,Dessert Shop,Tea Room,Bookstore,Burger Joint,Café,Champagne Bar,Chocolate Shop,Church,Exhibit,Scenic Lookout,Gift Shop
3,Tour Eiffel,48.858370,2.294481,0,Hotel,French Restaurant,Monument / Landmark,Champagne Bar,Gift Shop,Theme Park Ride / Attraction,Italian Restaurant,Art Museum,Nightclub,Plaza,Bagel Shop,Bookstore,Burger Joint,Café,Tea Room,Chocolate Shop,Church,Dessert Shop,Exhibit,Steakhouse


In [27]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=13)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(paris_merged['Latitude'], paris_merged['Longitude'], paris_merged['Lieux'], paris_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [28]:
# Cluster 0 == Musée du louvre
paris_merged.loc[paris_merged['Cluster Labels'] == 0, paris_merged.columns[[1] + list(range(5, paris_merged.shape[1]))]]

,Latitude,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
0,48.860611,Exhibit,Art Museum,Tea Room,Shopping Mall,Café,Chocolate Shop,French Restaurant,Bagel Shop,Bookstore,Burger Joint,Champagne Bar,Church,Dessert Shop,Vegetarian / Vegan Restaurant,Gift Shop,Theme Park Ride / Attraction,Ice Cream Shop,Italian Restaurant,Japanese Restaurant
3,48.858370,French Restaurant,Monument / Landmark,Champagne Bar,Gift Shop,Theme Park Ride / Attraction,Italian Restaurant,Art Museum,Nightclub,Plaza,Bagel Shop,Bookstore,Burger Joint,Café,Tea Room,Chocolate Shop,Church,Dessert Shop,Exhibit,Steakhouse


In [29]:
# Cluster 1 == Arc de triomphe
paris_merged.loc[paris_merged['Cluster Labels'] == 1, paris_merged.columns[[1] + list(range(5, paris_merged.shape[1]))]]

,Latitude,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
2,48.873792,Plaza,Bagel Shop,French Restaurant,Monument / Landmark,Nightclub,Vegetarian / Vegan Restaurant,Roof Deck,Steakhouse,Dessert Shop,Tea Room,Bookstore,Burger Joint,Café,Champagne Bar,Chocolate Shop,Church,Exhibit,Scenic Lookout,Gift Shop


In [30]:
# Cluster 2 == Tour Eiffel
paris_merged.loc[paris_merged['Cluster Labels'] == 2, paris_merged.columns[[1] + list(range(5, paris_merged.shape[1]))]]

,Latitude,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
1,48.852968,Bookstore,Burger Joint,Scenic Lookout,Plaza,Japanese Restaurant,Church,Ice Cream Shop,Dessert Shop,Vegetarian / Vegan Restaurant,Bagel Shop,Café,Champagne Bar,Chocolate Shop,Exhibit,Hotel,Gift Shop,Theme Park Ride / Attraction,Italian Restaurant,Monument / Landmark
